In [32]:
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import datetime as dt
import kagglehub as k
import json

imdb_movies_by_genre_df = pd.read_csv('Resources/imdb_movies_by_genre_dataset.csv')
movies_box_office_2000_to_2024_df = pd.read_csv('Resources/movies_box_office_2000_to_2024.csv')
mojo_budget_update_df = pd.read_csv('Resources/Mojo_budget_update.csv')
movies_box_office_2000_to_2024_df = pd.read_csv('Resources/movies_box_office_2000_to_2024.csv')
mojo_budget_data_df = pd.read_csv('Resources/Mojo_budget_data.csv')

imdb_movies_by_genre_df.head()




,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN


In [33]:
movies_box_office_2000_to_2024_df.head()


,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN


In [34]:
movies_box_office_2000_to_2024_df.columns

Index(['movie_id', 'movie_name', 'year', 'certificate', 'runtime', 'genre',
       'rating', 'description', 'director', 'director_id', 'star', 'star_id',
       'votes', 'gross(in $)'],
      dtype='object')

In [35]:
movies_box_office_2000_to_2024_df.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN


In [36]:
#Explore 'mojo_budget_update.csv'

In [37]:
mojo_budget_update_df.head()

,movie_id,title,year,trivia,mpaa,release_date,run_time,distributor,director,writer,...,main_actor_4,budget,domestic,international,worldwide,genre_1,genre_2,genre_3,genre_4,html
0,tt0099088,Back to the Future Part III,1990,"Stranded in 1955, Marty McFly learns about the...",NaN,May 25,1 hr 58 min,Universal Pictures,Robert Zemeckis,Robert Zemeckis,...,Thomas F. Wilson,40000000.0,88277583.0,157866667.0,246144250.0,Adventure,Comedy,Sci-Fi,Western,https://www.boxofficemojo.com/title/tt0099088/...
1,tt0099165,The Bonfire of the Vanities,1990,"After his mistress runs over a young teen, a W...",R,December 22,2 hr 5 min,Warner Bros.,Brian De Palma,Michael Cristofer,...,Kim Cattrall,47000000.0,15691192.0,NaN,15691192.0,Comedy,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0099165/...
2,tt0099348,Dances with Wolves,1990,"Lieutenant John Dunbar, assigned to a remote w...",NaN,November 9,3 hr 1 min,Orion Pictures,Kevin Costner,Michael Blake,...,Rodney A. Grant,22000000.0,184208848.0,240000000.0,424208848.0,Adventure,Drama,Western,NaN,https://www.boxofficemojo.com/title/tt0099348/...
3,tt0099422,Dick Tracy,1990,The comic strip detective finds his life vastl...,NaN,June 15,1 hr 45 min,Walt Disney Studios Motion Pictures,Warren Beatty,Chester Gould,...,Charlie Korsmo,47000000.0,103738726.0,59000000.0,162738726.0,Action,Comedy,Crime,Music,https://www.boxofficemojo.com/title/tt0099422/...
4,tt0099423,Die Hard 2,1990,John McClane attempts to avert disaster as rog...,NaN,July 6,2 hr 4 min,Twentieth Century Fox,Renny Harlin,Walter Wager,...,Reginald VelJohnson,70000000.0,117540947.0,122706486.0,240247433.0,Action,Thriller,NaN,NaN,https://www.boxofficemojo.com/title/tt0099423/...


In [38]:
mojo_budget_update_df.value_counts('year', ascending = False)

year
2010    174
2009    166
1999    160
2002    159
2005    158
2004    155
2000    147
2001    147
2011    146
2003    145
2006    143
2008    131
2013    126
2016    119
2014    119
2019    114
2015    113
2012    113
2017     99
2018     97
2007     94
1998     81
1997     70
1994     51
1991     47
1996     40
1995     38
1992     29
1993     29
1990     19
2020     14
Name: count, dtype: int64

In [39]:
mojo_budget_update_df.sort_values(by='year')


,movie_id,title,year,trivia,mpaa,release_date,run_time,distributor,director,writer,...,main_actor_4,budget,domestic,international,worldwide,genre_1,genre_2,genre_3,genre_4,html
0,tt0099088,Back to the Future Part III,1990,"Stranded in 1955, Marty McFly learns about the...",NaN,May 25,1 hr 58 min,Universal Pictures,Robert Zemeckis,Robert Zemeckis,...,Thomas F. Wilson,40000000.0,88277583.0,157866667.0,246144250.0,Adventure,Comedy,Sci-Fi,Western,https://www.boxofficemojo.com/title/tt0099088/...
18,tt0100802,Total Recall,1990,When a man goes for virtual vacation memories ...,NaN,June 1,1 hr 53 min,Sony Pictures Releasing,Paul Verhoeven,Philip K. Dick,...,Rachel Ticotin,65000000.0,119412921.0,141905000.0,261317921.0,Action,Sci-Fi,Thriller,NaN,https://www.boxofficemojo.com/title/tt0100802/...
17,tt0100758,Teenage Mutant Ninja Turtles,1990,Four teenage mutant ninja turtles emerge from ...,NaN,March 30,1 hr 33 min,New Line Cinema,Steve Barron,Kevin Eastman,...,David Forman,13500000.0,135265915.0,66700000.0,201965915.0,Action,Adventure,Comedy,Family,https://www.boxofficemojo.com/title/tt0100758/...
16,tt0100405,Pretty Woman,1990,A man in a legal but hurtful business needs an...,R,March 23,1 hr 59 min,Walt Disney Studios Motion Pictures,Garry Marshall,J.F. Lawton,...,Laura San Giacomo,14000000.0,178406268.0,285000000.0,463406268.0,Comedy,Romance,NaN,NaN,https://www.boxofficemojo.com/title/tt0100405/...
15,tt0100332,Paris Is Burning,1990,A chronicle of New York's drag scene in the 19...,R,August 9,1 hr 11 min,Miramax,Jennie Livingston,NaN,...,Paris Duprée,500000.0,3905297.0,NaN,3905297.0,Documentary,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt0100332/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2534,tt1634106,Bloodshot,2020,"Ray Garrison, a slain soldier, is re-animated ...",PG-13,March 6,1 hr 49 min,Sony Pictures Releasing,Dave Wilson,Jeff Wadlow,...,Toby Kebbell,45000000.0,10021787.0,18407068.0,28428855.0,Action,Drama,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt1634106/...
2989,tt3794354,Sonic the Hedgehog,2020,"After discovering a small, blue, fast hedgehog...",PG,February 12,1 hr 39 min,Paramount Pictures,Jeff Fowler,Patrick Casey,...,Tika Sumpter,85000000.0,146066470.0,160700000.0,306766470.0,Action,Adventure,Comedy,Family,https://www.boxofficemojo.com/title/tt3794354/...
2433,tt1502397,Bad Boys for Life,2020,Miami detectives Mike Lowrey and Marcus Burnet...,R,January 15,2 hr 4 min,Sony Pictures Releasing,Adil El Arbi,Peter Craig,...,Alexander Ludwig,90000000.0,204417855.0,214656791.0,419074646.0,Action,Comedy,Crime,Thriller,https://www.boxofficemojo.com/title/tt1502397/...
3228,tt8244784,The Hunt,2020,Twelve strangers wake up in a clearing. They d...,R,March 13,1 hr 30 min,Universal Pictures,Craig Zobel,Nick Cuse,...,Wayne Duvall,14000000.0,5812500.0,700000.0,6512500.0,Action,Horror,Thriller,NaN,https://www.boxofficemojo.com/title/tt8244784/...


In [40]:
mojo_budget_update_df.dtypes

movie_id            object
title               object
year                 int64
trivia              object
mpaa                object
release_date        object
run_time            object
distributor         object
director            object
writer              object
producer            object
composer            object
cinematographer     object
main_actor_1        object
main_actor_2        object
main_actor_3        object
main_actor_4        object
budget             float64
domestic           float64
international      float64
worldwide          float64
genre_1             object
genre_2             object
genre_3             object
genre_4             object
html                object
dtype: object

In [41]:
mojo_budget_update_df.columns

Index(['movie_id', 'title', 'year', 'trivia', 'mpaa', 'release_date',
       'run_time', 'distributor', 'director', 'writer', 'producer', 'composer',
       'cinematographer', 'main_actor_1', 'main_actor_2', 'main_actor_3',
       'main_actor_4', 'budget', 'domestic', 'international', 'worldwide',
       'genre_1', 'genre_2', 'genre_3', 'genre_4', 'html'],
      dtype='object')

In [42]:
mojo_budget_update_2000_to_2020_df = mojo_budget_update_df[(mojo_budget_update_df['year']== 2000) | (mojo_budget_update_df['year'] == 2020)]
mojo_budget_update_2000_to_2020_df

,movie_id,title,year,trivia,mpaa,release_date,run_time,distributor,director,writer,...,main_actor_4,budget,domestic,international,worldwide,genre_1,genre_2,genre_3,genre_4,html
356,tt0120630,Chicken Run,2000,When a cockerel apparently flies into a chicke...,NaN,June 23,1 hr 24 min,DreamWorks Distribution,Peter Lord,Peter Lord,...,Lynn Ferguson,45000000.0,106834564.0,118000000.0,224834564.0,Adventure,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...
387,tt0120755,Mission: Impossible II,2000,IMF agent Ethan Hunt is sent to Sydney to find...,PG-13,May 24,2 hr 3 min,Paramount Pictures,John Woo,Bruce Geller,...,Ving Rhames,125000000.0,215409889.0,330978219.0,546388108.0,Action,Adventure,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120755/...
413,tt0120903,X-Men,2000,In a world where mutants evolved super-powered...,PG-13,July 13,1 hr 44 min,Twentieth Century Fox,Bryan Singer,Tom DeSanto,...,Famke Janssen,75000000.0,157299718.0,139039810.0,296339528.0,Action,Adventure,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt0120903/...
417,tt0120913,Titan A.E.,2000,A young man learns that he has to find a hidde...,PG,June 16,1 hr 34 min,Twentieth Century Fox,Don Bluth,Hans Bauer,...,Jim Breuer,75000000.0,22753426.0,14001208.0,36754634.0,Action,Adventure,Animation,Family,https://www.boxofficemojo.com/title/tt0120913/...
419,tt0120917,The Emperor's New Groove,2000,Emperor Kuzco is turned into a llama by his ex...,NaN,December 15,1 hr 18 min,Walt Disney Studios Motion Pictures,Mark Dindal,Chris Williams,...,Patrick Warburton,100000000.0,89302687.0,80025000.0,169327687.0,Adventure,Animation,Comedy,Family,https://www.boxofficemojo.com/title/tt0120917/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208,tt7545266,Like a Boss,2020,Two friends with very different ideals start a...,R,January 9,1 hr 23 min,Paramount Pictures,Miguel Arteta,Sam Pitman,...,Destiny Adams,29000000.0,22169514.0,7583629.0,29753143.0,Comedy,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt7545266/...
3214,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,"After splitting with the Joker, Harley Quinn j...",R,February 5,1 hr 49 min,Warner Bros.,Cathy Yan,Christina Hodson,...,Jurnee Smollett-Bell,84500000.0,84158461.0,117700000.0,201858461.0,Action,Adventure,Crime,NaN,https://www.boxofficemojo.com/title/tt7713068/...
3218,tt7798646,The Photograph,2020,A series of intertwining love stories set in t...,PG-13,February 14,1 hr 46 min,Universal Pictures,Stella Meghie,Stella Meghie,...,Issa Rae,16000000.0,20578185.0,75019.0,20653204.0,Drama,Romance,NaN,NaN,https://www.boxofficemojo.com/title/tt7798646/...
3228,tt8244784,The Hunt,2020,Twelve strangers wake up in a clearing. They d...,R,March 13,1 hr 30 min,Universal Pictures,Craig Zobel,Nick Cuse,...,Wayne Duvall,14000000.0,5812500.0,700000.0,6512500.0,Action,Horror,Thriller,NaN,https://www.boxofficemojo.com/title/tt8244784/...


In [43]:
#'movie_id', 'title', 'year', 'trivia', 'mpaa', 'release_date',
      # 'run_time', 'distributor', 'director', 'writer', 'producer', 'composer',
       #'cinematographer', 'main_actor_1', 'main_actor_2', 'main_actor_3',
       #'main_actor_4', 'budget', 'domestic', 'international', 'worldwide',
       #'genre_1', 'genre_2', 'genre_3', 'genre_4', 'html'
       
clean_mojo_budget_update_2000_2020_df = mojo_budget_update_2000_to_2020_df[["movie_id", "title", "year", "main_actor_1", "main_actor_2", "domestic", "mpaa", "genre_1", "release_date"]]
clean_mojo_budget_update_2000_2020_df

,movie_id,title,year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
356,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564.0,NaN,Adventure,June 23
387,tt0120755,Mission: Impossible II,2000,Tom Cruise,Dougray Scott,215409889.0,PG-13,Action,May 24
413,tt0120903,X-Men,2000,Patrick Stewart,Hugh Jackman,157299718.0,PG-13,Action,July 13
417,tt0120913,Titan A.E.,2000,Matt Damon,Drew Barrymore,22753426.0,PG,Action,June 16
419,tt0120917,The Emperor's New Groove,2000,David Spade,John Goodman,89302687.0,NaN,Adventure,December 15
...,...,...,...,...,...,...,...,...,...
3208,tt7545266,Like a Boss,2020,Tiffany Haddish,Rose Byrne,22169514.0,R,Comedy,January 9
3214,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Margot Robbie,Rosie Perez,84158461.0,R,Action,February 5
3218,tt7798646,The Photograph,2020,Chelsea Peretti,LaKeith Stanfield,20578185.0,PG-13,Drama,February 14
3228,tt8244784,The Hunt,2020,Betty Gilpin,Hilary Swank,5812500.0,R,Action,March 13


In [44]:
rename_mojo_budget_update_2000_2020_df = clean_mojo_budget_update_2000_2020_df.rename(columns={'title':'movie_title', 'year':'movie_year'})
rename_mojo_budget_update_2000_2020_df

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
356,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564.0,NaN,Adventure,June 23
387,tt0120755,Mission: Impossible II,2000,Tom Cruise,Dougray Scott,215409889.0,PG-13,Action,May 24
413,tt0120903,X-Men,2000,Patrick Stewart,Hugh Jackman,157299718.0,PG-13,Action,July 13
417,tt0120913,Titan A.E.,2000,Matt Damon,Drew Barrymore,22753426.0,PG,Action,June 16
419,tt0120917,The Emperor's New Groove,2000,David Spade,John Goodman,89302687.0,NaN,Adventure,December 15
...,...,...,...,...,...,...,...,...,...
3208,tt7545266,Like a Boss,2020,Tiffany Haddish,Rose Byrne,22169514.0,R,Comedy,January 9
3214,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Margot Robbie,Rosie Perez,84158461.0,R,Action,February 5
3218,tt7798646,The Photograph,2020,Chelsea Peretti,LaKeith Stanfield,20578185.0,PG-13,Drama,February 14
3228,tt8244784,The Hunt,2020,Betty Gilpin,Hilary Swank,5812500.0,R,Action,March 13


In [45]:
rename_mojo_budget_update_2000_2020_df.isnull().sum()

movie_id         0
movie_title      0
movie_year       0
main_actor_1     0
main_actor_2     0
domestic         0
mpaa            10
genre_1          0
release_date     0
dtype: int64

In [46]:
rename_mojo_budget_update_2000_2020_df.dropna().copy()

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,release_date
387,tt0120755,Mission: Impossible II,2000,Tom Cruise,Dougray Scott,215409889.0,PG-13,Action,May 24
413,tt0120903,X-Men,2000,Patrick Stewart,Hugh Jackman,157299718.0,PG-13,Action,July 13
417,tt0120913,Titan A.E.,2000,Matt Damon,Drew Barrymore,22753426.0,PG,Action,June 16
423,tt0122459,Return to Me,2000,David Duchovny,Minnie Driver,32722798.0,PG,Comedy,April 7
454,tt0130623,Dinosaur,2000,D.B. Sweeney,Julianna Margulies,137748063.0,PG,Adventure,May 19
...,...,...,...,...,...,...,...,...,...
3208,tt7545266,Like a Boss,2020,Tiffany Haddish,Rose Byrne,22169514.0,R,Comedy,January 9
3214,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Margot Robbie,Rosie Perez,84158461.0,R,Action,February 5
3218,tt7798646,The Photograph,2020,Chelsea Peretti,LaKeith Stanfield,20578185.0,PG-13,Drama,February 14
3228,tt8244784,The Hunt,2020,Betty Gilpin,Hilary Swank,5812500.0,R,Action,March 13


In [47]:
#Explore 'mojo_budget_data.csv' file

In [48]:
mojo_budget_data_df


,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
0,tt0118589,Glitter,2001,Vondie Curtis-Hall,Cheryl L. West,Laurence Mark,Terence Blanchard,Geoffrey Simpson,Mariah Carey,Eric Benét,...,4274407,997259,5271666,PG-13,1 hr 44 min,Drama,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0118589/...
1,tt0120630,Chicken Run,2000,Peter Lord,Peter Lord,Peter Lord,Harry Gregson-Williams,Simon Jacobs,Mel Gibson,Julia Sawalha,...,106834564,118000000,224834564,NaN,1 hr 24 min,Adventure,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...
2,tt0120667,Fantastic Four,2005,Tim Story,Mark Frost,Avi Arad,John Ottman,Oliver Wood,Ioan Gruffudd,Michael Chiklis,...,154696080,178839854,333535934,PG-13,1 hr 46 min,Action,Adventure,Family,Fantasy,https://www.boxofficemojo.com/title/tt0120667/...
3,tt0120679,Frida,2002,Julie Taymor,Hayden Herrera,Lindsay Flickinger,Elliot Goldenthal,Rodrigo Prieto,Salma Hayek,Alfred Molina,...,25885000,30413474,56298474,R,2 hr 3 min,Biography,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0120679/...
4,tt0120681,From Hell,2001,Albert Hughes,Alan Moore,Jane Hamsher,Trevor Jones,Peter Deming,Johnny Depp,Heather Graham,...,31602566,42955549,74558115,R,2 hr 2 min,Horror,Mystery,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120681/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,tt8688634,21 Bridges,2019,Brian Kirk,Adam Mervis,Chadwick Boseman,Alex Belcher,Paul Cameron,Chadwick Boseman,Sienna Miller,...,28539757,21400000,49939757,R,1 hr 39 min,Action,Crime,Drama,Thriller,https://www.boxofficemojo.com/title/tt8688634/...
2472,tt8946378,Knives Out,2019,Rian Johnson,Rian Johnson,Ram Bergman,Nathan Johnson,Steve Yedlin,Daniel Craig,Chris Evans,...,165363234,146189956,311553190,PG-13,2 hr 11 min,Comedy,Crime,Drama,Mystery,https://www.boxofficemojo.com/title/tt8946378/...
2473,tt9024106,Unplanned,2019,Chuck Konzelman,Abby Johnson,Chris Jones,Stephen Blake Kanicka,Drew Maw,Ashley Bratcher,Brooks Ryan,...,19005109,2046806,21051915,R,1 hr 49 min,Biography,Drama,NaN,NaN,https://www.boxofficemojo.com/title/tt9024106/...
2474,tt9134216,Playing with Fire,2019,Andy Fickman,Dan Ewen,Todd Garner,Nathan Wang,Dean Semler,John Cena,Keegan-Michael Key,...,44451847,24179822,68631669,PG,1 hr 36 min,Comedy,Family,NaN,NaN,https://www.boxofficemojo.com/title/tt9134216/...


In [49]:
mojo_budget_data_df.value_counts('movie_year', ascending= False)

movie_year
2010    163
2002    151
2009    149
2004    142
2011    140
2001    138
2000    137
2005    137
2003    134
2006    126
2008    123
2013    122
2014    117
2016    117
2015    109
2012    104
2017     98
2018     92
2007     88
2019     75
2020     14
Name: count, dtype: int64

In [50]:
mojo_budget_data_df.sort_values(by= 'movie_year')

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
109,tt0185183,Battlefield Earth,2000,Roger Christian,Corey Mandell,Jonathan D. Krane,Elia Cmiral,Giles Nuttgens,John Travolta,Forest Whitaker,...,21471685,8253978,29725663,PG-13,1 hr 58 min,Action,Adventure,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt0185183/...
101,tt0183505,"Me, Myself & Irene",2000,Bobby Farrelly,Peter Farrelly,Bobby Farrelly,Lee Scott,Mark Irwin,Jim Carrey,Renée Zellweger,...,90570999,58700000,149270999,R,1 hr 56 min,Comedy,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt0183505/...
102,tt0183523,Mission to Mars,2000,Brian De Palma,Lowell Cannon,Tom Jacobson,Ennio Morricone,Stephen H. Burum,Tim Robbins,Gary Sinise,...,60883407,50100000,110983407,PG,1 hr 54 min,Adventure,Sci-Fi,Thriller,NaN,https://www.boxofficemojo.com/title/tt0183523/...
172,tt0210616,Center Stage,2000,Nicholas Hytner,Carol Heikkinen,Laurence Mark,George Fenton,Geoffrey Simpson,Amanda Schull,Ethan Stiefel,...,17200925,9185016,26385941,PG-13,1 hr 55 min,Drama,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0210616/...
171,tt0210567,Beautiful,2000,Sally Field,Jon Bernstein,John Bertolli,John Frizzell,Robert D. Yeoman,Minnie Driver,Hallie Eisenberg,...,3157348,12582,3169930,PG-13,1 hr 52 min,Comedy,Drama,NaN,NaN,https://www.boxofficemojo.com/title/tt0210567/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828,tt1634106,Bloodshot,2020,Dave Wilson,Jeff Wadlow,Toby Jaffe,Steve Jablonsky,Jacques Jouffret,Vin Diesel,Eiza González,...,10021787,19727667,29749454,PG-13,1 hr 49 min,Action,Drama,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt1634106/...
1310,tt0983946,Fantasy Island,2020,Jeff Wadlow,Jillian Jacobs,Jason Blum,Bear McCreary,Toby Oliver,Michael Peña,Maggie Q,...,26441782,22165774,48607556,PG-13,1 hr 49 min,Adventure,Fantasy,Horror,Mystery,https://www.boxofficemojo.com/title/tt0983946/...
2416,tt6673612,Dolittle,2020,Stephen Gaghan,Stephen Gaghan,Susan Downey,Danny Elfman,Guillermo Navarro,Robert Downey Jr.,Antonio Banderas,...,77047065,150861065,227908130,PG,1 hr 41 min,Adventure,Comedy,Family,Fantasy,https://www.boxofficemojo.com/title/tt6673612/...
1371,tt1051906,The Invisible Man,2020,Leigh Whannell,Leigh Whannell,Jason Blum,Benjamin Wallfisch,Stefan Duscio,Elisabeth Moss,Oliver Jackson-Cohen,...,64914050,59647175,124561225,R,2 hr 4 min,Horror,Mystery,Sci-Fi,Thriller,https://www.boxofficemojo.com/title/tt1051906/...


In [51]:
mojo_budget_data_df.dtypes

movie_id           object
movie_title        object
movie_year          int64
director           object
writer             object
producer           object
composer           object
cinematographer    object
main_actor_1       object
main_actor_2       object
main_actor_3       object
main_actor_4       object
budget              int64
domestic            int64
international       int64
worldwide           int64
mpaa               object
run_time           object
genre_1            object
genre_2            object
genre_3            object
genre_4            object
link               object
dtype: object

In [52]:
mojo_budget_data_df.columns

Index(['movie_id', 'movie_title', 'movie_year', 'director', 'writer',
       'producer', 'composer', 'cinematographer', 'main_actor_1',
       'main_actor_2', 'main_actor_3', 'main_actor_4', 'budget', 'domestic',
       'international', 'worldwide', 'mpaa', 'run_time', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'link'],
      dtype='object')

In [53]:
mojo_budget_data_2000_to_2020_df = mojo_budget_data_df[(mojo_budget_data_df['movie_year']== 2000) | (mojo_budget_data_df['movie_year'] == 2020)]
mojo_budget_data_2000_to_2020_df

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
1,tt0120630,Chicken Run,2000,Peter Lord,Peter Lord,Peter Lord,Harry Gregson-Williams,Simon Jacobs,Mel Gibson,Julia Sawalha,...,106834564,118000000,224834564,NaN,1 hr 24 min,Adventure,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...
6,tt0120755,Mission: Impossible II,2000,John Woo,Bruce Geller,Tom Cruise,Hans Zimmer,Jeffrey L. Kimball,Tom Cruise,Dougray Scott,...,215409889,330978219,546388108,PG-13,2 hr 3 min,Action,Adventure,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120755/...
9,tt0120903,X-Men,2000,Bryan Singer,Tom DeSanto,Lauren Shuler Donner,Michael Kamen,Newton Thomas Sigel,Patrick Stewart,Hugh Jackman,...,157299718,139039810,296339528,PG-13,1 hr 44 min,Action,Adventure,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt0120903/...
11,tt0120913,Titan A.E.,2000,Don Bluth,Hans Bauer,Don Bluth,Graeme Revell,NaN,Matt Damon,Drew Barrymore,...,22753426,14001208,36754634,PG,1 hr 34 min,Action,Adventure,Animation,Family,https://www.boxofficemojo.com/title/tt0120913/...
12,tt0120917,The Emperor's New Groove,2000,Mark Dindal,Chris Williams,Randy Fullmer,John Debney,Thomas Baker,David Spade,John Goodman,...,89302687,80025000,169327687,NaN,1 hr 18 min,Adventure,Animation,Comedy,Family,https://www.boxofficemojo.com/title/tt0120917/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,tt7545266,Like a Boss,2020,Miguel Arteta,Sam Pitman,Marc Evans,Christophe Beck,Jas Shelton,Rose Byrne,Salma Hayek,...,22169514,7583629,29753143,R,1 hr 23 min,Comedy,NaN,NaN,NaN,https://www.boxofficemojo.com/title/tt7545266/...
2455,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Cathy Yan,Christina Hodson,Sue Kroll,Daniel Pemberton,Matthew Libatique,Margot Robbie,Rosie Perez,...,84158461,117700000,201858461,R,1 hr 49 min,Action,Adventure,Crime,NaN,https://www.boxofficemojo.com/title/tt7713068/...
2458,tt7798646,The Photograph,2020,Stella Meghie,Stella Meghie,James Lopez,Robert Glasper,Mark Schwartzbard,LaKeith Stanfield,Chelsea Peretti,...,20578185,70122,20648307,PG-13,1 hr 46 min,Drama,Romance,NaN,NaN,https://www.boxofficemojo.com/title/tt7798646/...
2467,tt8244784,The Hunt,2020,Craig Zobel,Nick Cuse,Jason Blum,Nathan Barr,Darran Tiernan,Betty Gilpin,Hilary Swank,...,5812500,700000,6512500,R,1 hr 30 min,Action,Horror,Thriller,NaN,https://www.boxofficemojo.com/title/tt8244784/...


In [54]:
clean_mojo_budget_data_2000_2020_df = mojo_budget_data_2000_to_2020_df[["movie_id", "movie_title", "movie_year", "main_actor_1", "main_actor_2", "domestic", "mpaa", "genre_1"]]
clean_mojo_budget_data_2000_2020_df

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1
1,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564,NaN,Adventure
6,tt0120755,Mission: Impossible II,2000,Tom Cruise,Dougray Scott,215409889,PG-13,Action
9,tt0120903,X-Men,2000,Patrick Stewart,Hugh Jackman,157299718,PG-13,Action
11,tt0120913,Titan A.E.,2000,Matt Damon,Drew Barrymore,22753426,PG,Action
12,tt0120917,The Emperor's New Groove,2000,David Spade,John Goodman,89302687,NaN,Adventure
...,...,...,...,...,...,...,...,...
2449,tt7545266,Like a Boss,2020,Rose Byrne,Salma Hayek,22169514,R,Comedy
2455,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Margot Robbie,Rosie Perez,84158461,R,Action
2458,tt7798646,The Photograph,2020,LaKeith Stanfield,Chelsea Peretti,20578185,PG-13,Drama
2467,tt8244784,The Hunt,2020,Betty Gilpin,Hilary Swank,5812500,R,Action


In [55]:
clean_mojo_budget_data_2000_2020_df.isnull().sum()

movie_id        0
movie_title     0
movie_year      0
main_actor_1    0
main_actor_2    0
domestic        0
mpaa            8
genre_1         0
dtype: int64

In [56]:
clean_mojo_budget_data_2000_2020_df.dropna().copy()

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1
6,tt0120755,Mission: Impossible II,2000,Tom Cruise,Dougray Scott,215409889,PG-13,Action
9,tt0120903,X-Men,2000,Patrick Stewart,Hugh Jackman,157299718,PG-13,Action
11,tt0120913,Titan A.E.,2000,Matt Damon,Drew Barrymore,22753426,PG,Action
15,tt0122459,Return to Me,2000,David Duchovny,Minnie Driver,32722798,PG,Comedy
18,tt0130623,Dinosaur,2000,D.B. Sweeney,Julianna Margulies,137748063,PG,Adventure
...,...,...,...,...,...,...,...,...
2449,tt7545266,Like a Boss,2020,Rose Byrne,Salma Hayek,22169514,R,Comedy
2455,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Margot Robbie,Rosie Perez,84158461,R,Action
2458,tt7798646,The Photograph,2020,LaKeith Stanfield,Chelsea Peretti,20578185,PG-13,Drama
2467,tt8244784,The Hunt,2020,Betty Gilpin,Hilary Swank,5812500,R,Action


In [58]:
#We need to merge the two datasets. 
merge_df = pd.merge(clean_mojo_budget_data_2000_2020_df, clean_mojo_budget_update_2000_2020_df)
merge_df

,movie_id,movie_title,movie_year,main_actor_1,main_actor_2,domestic,mpaa,genre_1,title,year,release_date
0,tt0120630,Chicken Run,2000,Mel Gibson,Julia Sawalha,106834564,NaN,Adventure,Chicken Run,2000,June 23
1,tt0120755,Mission: Impossible II,2000,Tom Cruise,Dougray Scott,215409889,PG-13,Action,Mission: Impossible II,2000,May 24
2,tt0120903,X-Men,2000,Patrick Stewart,Hugh Jackman,157299718,PG-13,Action,X-Men,2000,July 13
3,tt0120913,Titan A.E.,2000,Matt Damon,Drew Barrymore,22753426,PG,Action,Titan A.E.,2000,June 16
4,tt0120917,The Emperor's New Groove,2000,David Spade,John Goodman,89302687,NaN,Adventure,The Emperor's New Groove,2000,December 15
...,...,...,...,...,...,...,...,...,...,...,...
144,tt7504726,The Call of the Wild,2020,Harrison Ford,Omar Sy,62342368,PG,Adventure,The Call of the Wild,2020,January 3
145,tt7510346,The Turning,2020,Mackenzie Davis,Finn Wolfhard,15472775,PG-13,Drama,The Turning,2020,January 22
146,tt7713068,Birds of Prey: And the Fantabulous Emancipatio...,2020,Margot Robbie,Rosie Perez,84158461,R,Action,Birds of Prey: And the Fantabulous Emancipatio...,2020,February 5
147,tt8244784,The Hunt,2020,Betty Gilpin,Hilary Swank,5812500,R,Action,The Hunt,2020,March 13


In [60]:
merge_df.value_counts('movie_title')

movie_title
102 Dalmatians         1
Shanghai Noon          1
Snow Day               1
Sonic the Hedgehog     1
Space Cowboys          1
                      ..
Here on Earth          1
High Fidelity          1
Highlander: Endgame    1
Hollow Man             1
X-Men                  1
Name: count, Length: 149, dtype: int64

In [62]:
merge_df.to_csv('merged_mojo_dataset', index=False)